<a href="https://colab.research.google.com/github/JoshuaBarsky/Projects/blob/main/OSPF_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**OSPF program simulation.**

This is a program that simulates OSPF between two routers.



> "OSPF will listen to neighbors and gather all link state data available to build a topology map of all available paths in its network and then save the information in its topology database"
>
> https://www.linkedin.com/pulse/ospf-routing-protocol-use-cases-surayya-shaikh/

**This program is mainly for demonstration purposes only.**










This program is designed around Object Oriented Programming.

There are two routers in this program as follows.


*   Router 1: R1
*   Router 2: R2

Each router contains the following information:


*   Router ID
*   Router Number
*   Router Mac address
*   Neighbor Table (neighbor Mac, neighbor IP)
*   Incoming comms (cache)





In this demonstration, R1 and R2 will learn of one another's Mac address and IP addresses via OSPF hello messages.

These hello messages are packaged with the following.


*   Ethernet header (source and destination Mac Address)
*   IP Header (source and destination IP Address)
*   Packet data
*   Packet type

The packet is structured as shown below.

##[source Mac, destination Mac, source IP, destination IP, message, OSPF packet type]

*Please note: in this demonstration packet type 1 is a hello message, while packet type 2 is an encrypted message. This does not represent actual packet types but was created for the purposes of this demonstration.*



###Encryption
The encrypted portion of the code demonstrates how encrypted messages can be sent via a network. This does not use actual asymmetric encryption since, as in real world applications, but it does use a private key based encryption method. Given both R1 and R2 have the same encryption key, encrypted messages can be shared between the two devices without unwanted peering eyes seeing what's being shared.

##**ATTN: In order to run the program, press the play button on the top-left of each code cell.**

**Do this in consecutive order (top to bottom)...the last code cell shows the resulting output.**

In [ ]:
import time
from cryptography.fernet import Fernet

packet = ["", "", "", "", "", 0] #ethernet header, ip header, ospf packet header, ospf packet type
key = Fernet.generate_key()

In [ ]:
class router:

  key = ""
  routerID = ""
  routerNumber = 0
  routerMac = ""
  IP_address = ""
  neighbor_table = ["UNKNOWN", "UNKNOWN"]
  incoming_comms = ["", "", "", "", "", 0]

  def __init__(self, routerID = ""):
    self.routerID = ""
    self.incoming_comms = ["", "", "", "", "", 0]

  def set_key(self, x):
    self.key = x

  def set_routerNumber(self, x):
    self.routerNumber = x

  def get_routerNumber(self):
    return self.routerNumber

  def set_routerMac(self, x):
    self.routerMac = x

  def get_routerMac(self):
    return self.routerMac

  def set_ID(self, x):
    self.routerID = x

  def get_ID(self):
    return self.routerID

  def set_IP(self, x):
    self.IP_address = x

  def get_IP(self):
    return self.IP_address

  def read_incoming_comms(self, x):
    self.incoming_comms = x
    if self.incoming_comms[5] == 1:
      print("{}: incoming packet {}".format(self.routerID, x))
      print("{}: Hello recieved.".format(self.routerID))
      self.neighbor_table[0] = x[0]
      self.neighbor_table[1] = x[2]
    elif self.incoming_comms[5] == 2:
      print("{}: incoming packet {}".format(self.routerID, x))
      print("{}: Encrypted message recieved.".format(self.routerID))
      self.neighbor_table[0] = x[0]
      self.neighbor_table[1] = x[2]
      print(self.decrypt_message(self.incoming_comms[4], key))
    elif self.incoming_comms[5] == 0:
      print("{}: No incoming packet.".format(self.routerID))

  def send_hello(self):
    self.incoming_comms = ["", "", "", "", "", 0] #clears incomming coms
    print("{}: Hello sent.".format(self.routerID))
    return ["{}".format(self.routerMac), "{}".format(self.neighbor_table[0]), "{}".format(self.IP_address), "{}".format(self.neighbor_table[1]), "", 1]


  def send_encrypted_message(self, x, key):
    fernet = Fernet(key)
    encMessage = fernet.encrypt(x.encode())
    print("{}: original string: {}".format(self.routerID, x))
    print("{}: encrypted string: {}".format(self.routerID, encMessage))
    print("{}: Encrypted message sent.".format(self.routerID))
    return ["{}".format(self.routerMac), "{}".format(self.neighbor_table[0]), "{}".format(self.IP_address), "{}".format(self.neighbor_table[1]), encMessage, 2]

  def decrypt_message(self, xx, key):
    fernet = Fernet(key)
    decMessage = fernet.decrypt(xx).decode()
    return "{}: decrypted string: {}".format(self.routerID, decMessage)

In [ ]:
R1 = router()
R1.set_routerMac("AAAA")
R1.set_routerNumber(1)
R1.set_ID("1.1.1.1")
R1.set_IP("10.10.1.1")
R1.set_key(key)
R2 = router()
R2.set_routerMac("BBBB")
R2.set_routerNumber(2)
R2.set_ID("2.2.2.2")
R2.set_IP("10.10.2.1")
R2.set_key(key)

In [ ]:
packet = ["", "", "", "", "", 0]

R1.read_incoming_comms(packet)
packet = R1.send_hello()
time.sleep(2)
print("---")

R2.read_incoming_comms(packet)
packet = R2.send_hello()

time.sleep(2)
print("---")

R1.read_incoming_comms(packet)
packet = R1.send_hello()

time.sleep(2)
print("---")

R2.read_incoming_comms(packet)
packet = R2.send_encrypted_message("This is a secret message!", key)

time.sleep(2)
print("---")

R1.read_incoming_comms(packet)

1.1.1.1: No incoming packet.
1.1.1.1: Hello sent.
---
2.2.2.2: incoming packet ['AAAA', 'UNKNOWN', '10.10.1.1', 'UNKNOWN', '', 1]
2.2.2.2: Hello recieved.
2.2.2.2: Hello sent.
---
1.1.1.1: incoming packet ['BBBB', 'AAAA', '10.10.2.1', '10.10.1.1', '', 1]
1.1.1.1: Hello recieved.
1.1.1.1: Hello sent.
---
2.2.2.2: incoming packet ['AAAA', 'BBBB', '10.10.1.1', '10.10.2.1', '', 1]
2.2.2.2: Hello recieved.
2.2.2.2: original string: This is a secret message!
2.2.2.2: encrypted string: b'gAAAAABkh00CxRVgEyz3JlxRDYcrevYJKZkgM3AbPt2YI-s5Xg2kKN8WVcSoOTHXjFpjiGnXX8A7JENHExG-xRsSzEUTwYQzBvZOz0pj3bwEttf9uGltRcg='
2.2.2.2: Encrypted message sent.
---
1.1.1.1: incoming packet ['BBBB', 'AAAA', '10.10.2.1', '10.10.1.1', b'gAAAAABkh00CxRVgEyz3JlxRDYcrevYJKZkgM3AbPt2YI-s5Xg2kKN8WVcSoOTHXjFpjiGnXX8A7JENHExG-xRsSzEUTwYQzBvZOz0pj3bwEttf9uGltRcg=', 2]
1.1.1.1: Encrypted message recieved.
1.1.1.1: decrypted string: This is a secret message!
